In [ ]:
import sys,os
import glob
import torch,dill
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from iresnet import iResBlock
from iEluNetwork import iConv2d
from spectral_norm import SN

In [ ]:
paths = glob.glob(os.path.expanduser('~/tb-experiments/iresnet_WTW/**/*.trainer'),recursive=True)
elu_paths = glob.glob(os.path.expanduser('~/tb-experiments/elu_flow_full/**/*.trainer'),recursive=True)
paths = elu_paths
print(paths)


In [ ]:
trainer = torch.load(paths[-1],pickle_module=dill)
trainer.model.eval();

In [ ]:
def change_tol(module):
    if isinstance(module,iResBlock):
        module.inverse_tol=1e-10
        print(module.iters_per_reverse)
    if isinstance(module,SN):
        print(module._s)
    if isinstance(module,iConv2d):
        print(module._reverse_iters)
        try: print(module.iters_per_reverse)
        except ZeroDivisionError: pass

In [ ]:
for mb in trainer.dataloaders['dev']:
    break
inp =x=mb[0][:1]
trainer.model.apply(change_tol);

In [ ]:
with torch.no_grad():
    output = trainer.model.body(inp)
    inp_rec = trainer.model.inverse(output)

In [ ]:
((inp_rec[0] - inp[0]).norm()/(inp[0]).norm())

In [ ]:
plt.hist(output.cpu().numpy().reshape(-1))

In [ ]:
with torch.no_grad():
    output = trainer.model[1].body(inp)

In [ ]:
sampled_imgs = trainer.model.sample(32).cpu().data

In [ ]:
grid = vutils.make_grid(sampled_imgs, normalize=True)
plt.imshow(grid.permute(1,2,0))

In [ ]:
grid.shape

In [ ]:
list(z_.shape for z_ in output)

In [ ]:
with torch.no_grad():
    inp_reconstructed = trainer.model[1].body.inverse(output);

In [ ]:
((inp_reconstructed[0] - inp[0]).norm()/(inp[0]).norm())

In [ ]:
plt.hist(inp_reconstructed[0].cpu().data.numpy().reshape(-1),alpha=.5)
plt.hist(inp[0].cpu().data.numpy().reshape(-1),alpha=.5)

In [ ]:
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
#print(y.shape)
grid = vutils.make_grid(mb[0]).cpu().data.numpy().transpose(1,2,0)
plt.imshow(grid)
plt.show()
grid = vutils.make_grid(inp_reconstructed).cpu().data.numpy().transpose(1,2,0)
plt.imshow(grid)
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)

In [ ]:
for zi in z: print(zi.shape)